## 1. Configuración Inicial

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import folium
from folium import plugins
import json

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Rutas
BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
VECTOR_DIR = DATA_DIR / 'vector'
OUTPUT_DIR = BASE_DIR / 'outputs' / 'figures'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Librerías cargadas correctamente")

## 2. Exportar Hotspots a GeoJSON

In [ ]:
# Ejecutar script de exportación
import sys
sys.path.append(str(BASE_DIR / 'scripts'))

from export_hotspots_arcgis import main as export_main

# Ejecutar exportación
output_file, validation_df = export_main()

if output_file:
    print(f"\n✅ GeoJSON exportado: {output_file}")
    print(f"✅ Plantilla CSV generada")
else:
    print("❌ Error en la exportación")

## 3. Visualización Previa de Hotspots

Antes de subir a ArcGIS, visualizamos los hotspots en un mapa interactivo:

In [ ]:
# Cargar GeoJSON exportado
geojson_file = PROCESSED_DIR / 'validacion_hotspots.geojson'
hotspots = gpd.read_file(geojson_file)

print(f"Hotspots cargados: {len(hotspots)}")
print(f"\nColumnas disponibles: {hotspots.columns.tolist()}")
print(f"\nPrimeras 5 zonas:")
display(hotspots.head())

In [ ]:
# Crear mapa interactivo con Folium
# Calcular centro del mapa
center = [hotspots.geometry.centroid.y.mean(), hotspots.geometry.centroid.x.mean()]

# Crear mapa base
m = folium.Map(
    location=center,
    zoom_start=12,
    tiles='OpenStreetMap'
)

# Agregar capa de satélite
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Satellite',
    overlay=False,
    control=True
).add_to(m)

# Agregar hotspots con popup
for idx, row in hotspots.iterrows():
    # Determinar columna de identificación
    if 'ID_Zona' in row:
        zona_id = row['ID_Zona']
    elif 'zona_id' in row:
        zona_id = row['zona_id']
    else:
        zona_id = f"Zona_{idx}"
    
    # Crear popup con información
    popup_html = f"""
    <div style="font-family: Arial; font-size: 12px;">
        <b>Zona:</b> {zona_id}<br>
    """
    
    if 'Urbanizacion_ha' in row:
        popup_html += f"<b>Urbanización:</b> {row['Urbanizacion_ha']:.2f} ha<br>"
    if 'Indice_Transf' in row:
        popup_html += f"<b>Índice Transf:</b> {row['Indice_Transf']:.2f}<br>"
    
    popup_html += "</div>"
    
    # Agregar polígono
    folium.GeoJson(
        row.geometry,
        style_function=lambda x: {
            'fillColor': 'red',
            'color': 'darkred',
            'weight': 2,
            'fillOpacity': 0.4
        },
        popup=folium.Popup(popup_html, max_width=250)
    ).add_to(m)

# Agregar control de capas
folium.LayerControl().add_to(m)

# Mostrar mapa
m

## 4. Instrucciones para ArcGIS Online

### Paso a Paso:

1. **Acceder a ArcGIS Online:**
   - Ir a: `geo-usach.maps.arcgis.com`
   - Iniciar sesión con credenciales institucionales
   - Abrir **Map Viewer**

2. **Cargar el GeoJSON:**
   - Click en botón **"Agregar"** (símbolo +)
   - Seleccionar **"Agregar capa desde archivo"**
   - Subir archivo: `data/processed/validacion_hotspots.geojson`
   - Ajustar transparencia al 50% para ver imagen satelital debajo

3. **Agregar Capa Wayback:**
   - Click en **"Agregar"** > **"Examinar capas"**
   - Cambiar fuente a **"ArcGIS Online"** o **"Living Atlas"**
   - Buscar: **"World Imagery Wayback"**
   - Agregar capas de 2018 y 2024 para comparación

4. **Validar Visualmente:**
   - Hacer zoom a cada hotspot (polígono rojo)
   - Alternar entre imagen actual y Wayback 2018
   - Pregunta clave: *¿Hay construcciones nuevas aquí?*
   - Registrar observación en plantilla CSV

5. **Capturar Evidencia:**
   - Tomar screenshots de 5-10 casos representativos
   - Incluir: Verdaderos Positivos, Falsos Positivos (si hay), casos dudosos
   - Guardar en: `outputs/figures/validacion_arcgis/`

---

## 5. Cargar Resultados de Validación

**Después de completar la validación en ArcGIS, regresar aquí para analizar los resultados.**

In [ ]:
# Cargar plantilla CSV completada
validation_file = PROCESSED_DIR / 'plantilla_validacion_arcgis.csv'

# Verificar si ya fue completada
if validation_file.exists():
    validation_results = pd.read_csv(validation_file)
    
    # Verificar si hay datos completados
    completed = validation_results['Coincide'].notna().sum()
    
    if completed > 0:
        print(f"✅ Validación completada: {completed}/{len(validation_results)} zonas")
        display(validation_results.head(10))
    else:
        print("⚠️  Plantilla aún no completada. Completar validación en ArcGIS primero.")
        print(f"📁 Archivo: {validation_file}")
else:
    print("❌ Plantilla no encontrada. Ejecutar celda de exportación primero.")

## 6. Análisis de Resultados de Validación

**Calcular métricas de precisión basadas en la validación visual:**

In [ ]:
# Calcular matriz de confusión
if 'validation_results' in locals() and completed > 0:
    # Convertir respuestas a binario
    validation_results['Coincide_Bin'] = validation_results['Coincide'].str.lower().map({
        'sí': 1, 'si': 1, 'yes': 1, '✅': 1, 'x': 1,
        'no': 0, '❌': 0, '': None
    })
    
    # Filtrar solo validados
    validated = validation_results[validation_results['Coincide_Bin'].notna()]
    
    # Calcular métricas
    total = len(validated)
    verdaderos_positivos = (validated['Coincide_Bin'] == 1).sum()
    falsos_positivos = (validated['Coincide_Bin'] == 0).sum()
    
    precision = (verdaderos_positivos / total * 100) if total > 0 else 0
    
    print("=" * 60)
    print("   RESULTADOS DE VALIDACIÓN VISUAL")
    print("=" * 60)
    print(f"\nTotal zonas validadas: {total}")
    print(f"Verdaderos Positivos: {verdaderos_positivos} ({verdaderos_positivos/total*100:.1f}%)")
    print(f"Falsos Positivos: {falsos_positivos} ({falsos_positivos/total*100:.1f}%)")
    print(f"\n📊 PRECISIÓN GLOBAL: {precision:.1f}%")
    
    # Comparar con estándar USGS (85%)
    if precision >= 85:
        print(f"\n✅ Supera estándar USGS (85%)")
    else:
        print(f"\n⚠️  Por debajo de estándar USGS (85%)")
    
    print("=" * 60)
    
    # Visualizar matriz de confusión
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico de barras
    categories = ['Verdaderos\nPositivos', 'Falsos\nPositivos']
    values = [verdaderos_positivos, falsos_positivos]
    colors = ['#2ecc71', '#e74c3c']
    
    ax1.bar(categories, values, color=colors, alpha=0.7, edgecolor='black')
    ax1.set_ylabel('Número de Zonas')
    ax1.set_title('Matriz de Confusión - Validación Visual', fontsize=12, fontweight='bold')
    ax1.grid(axis='y', alpha=0.3)
    
    # Agregar valores en barras
    for i, v in enumerate(values):
        ax1.text(i, v + 0.5, str(v), ha='center', va='bottom', fontweight='bold')
    
    # Gráfico de pastel
    ax2.pie(values, labels=categories, autopct='%1.1f%%', colors=colors, 
            startangle=90, textprops={'fontsize': 11, 'weight': 'bold'})
    ax2.set_title(f'Distribución de Resultados\n(Precisión: {precision:.1f}%)', 
                  fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    
    # Guardar
    output_file = OUTPUT_DIR / 'validacion_arcgis_resultados.png'
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"\n💾 Figura guardada: {output_file}")
    
    plt.show()
    
else:
    print("⚠️  Completar validación en ArcGIS y llenar plantilla CSV primero.")

## 7. Análisis de Errores (Falsos Positivos)

**Identificar patrones en los casos donde no hubo concordancia:**

In [ ]:
if 'validated' in locals():
    # Filtrar falsos positivos
    falsos_pos = validated[validated['Coincide_Bin'] == 0]
    
    if len(falsos_pos) > 0:
        print(f"\n📋 ANÁLISIS DE FALSOS POSITIVOS ({len(falsos_pos)} casos):\n")
        print("="*80)
        
        for idx, row in falsos_pos.iterrows():
            print(f"\nZona: {row['ID_Zona']}")
            if 'Urbanizacion_ha' in row:
                print(f"  Urbanización detectada: {row['Urbanizacion_ha']:.2f} ha")
            print(f"  Observación: {row['Observacion_Visual_ArcGIS']}")
            print(f"  Comentarios: {row['Comentarios']}")
            print("-"*80)
        
        # Análisis de patrones
        print("\n🔍 POSIBLES CAUSAS DE ERROR:")
        print("  1. Confusión espectral (invernaderos, suelo desnudo)")
        print("  2. Cambios temporales no relacionados con urbanización")
        print("  3. Resolución espacial insuficiente (10m Sentinel-2)")
        print("  4. Umbrales de clasificación muy sensibles")
    else:
        print("\n✅ No se detectaron falsos positivos. Precisión del 100%!")

## 8. Tabla de Validación para Informe

**Generar tabla formateada para incluir en el informe final:**

In [ ]:
if 'validated' in locals():
    # Seleccionar 10 casos representativos para tabla en informe
    # 8 verdaderos positivos + 2 falsos positivos (si existen)
    vp = validated[validated['Coincide_Bin'] == 1].head(8)
    fp = validated[validated['Coincide_Bin'] == 0].head(2)
    
    tabla_informe = pd.concat([vp, fp])
    
    # Limpiar para LaTeX
    tabla_latex = tabla_informe[[
        'ID_Zona', 'Clasificacion_Python', 
        'Observacion_Visual_ArcGIS', 'Coincide'
    ]].copy()
    
    # Reemplazar símbolos
    tabla_latex['Coincide'] = tabla_latex['Coincide'].replace({
        'sí': '✓', 'si': '✓', 'yes': '✓', '✅': '✓',
        'no': '✗', '❌': '✗'
    })
    
    # Mostrar
    print("\n📊 TABLA PARA INFORME (Top 10 casos):")
    print("="*80)
    display(tabla_latex)
    
    # Exportar a LaTeX
    latex_output = tabla_latex.to_latex(
        index=False,
        caption='Validación visual de hotspots de urbanización mediante ArcGIS Online',
        label='tab:validacion_arcgis'
    )
    
    # Guardar
    latex_file = OUTPUT_DIR.parent / 'reports' / 'tabla_validacion_arcgis.tex'
    latex_file.parent.mkdir(parents=True, exist_ok=True)
    latex_file.write_text(latex_output)
    
    print(f"\n💾 Tabla LaTeX guardada: {latex_file}")

## 9. Conclusiones de la Validación

### Resumen Ejecutivo:

La validación visual mediante ArcGIS Online con imágenes de alta resolución (World Imagery) confirmó la robustez del método de detección de cambios implementado. Los resultados obtenidos:

- **Precisión global:** [COMPLETAR después de validación]%
- **Verdaderos positivos:** [COMPLETAR] de 20 zonas validadas
- **Cumplimiento estándar USGS:** [✓/✗] (requiere ≥85%)

Los casos de error identificados se explican principalmente por:
1. Confusión espectral con estructuras agrícolas (invernaderos)
2. Límites difusos en urbanización dispersa (<100m²)
3. Resolución espacial de Sentinel-2 (10m) vs detalle requerido

### Recomendaciones:

- Integrar imágenes de mayor resolución (≤3m) para urbanización dispersa
- Implementar máscaras de exclusión para invernaderos mediante NDVI temporal
- Validación adicional con catastro municipal de permisos de construcción

---

**Siguiente paso:** Incorporar estos resultados en Sección 4 del informe final.